In [1]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394484 sha256=ebdb5e0927c0b04f168ebe618d217fb7f15f9385c9ac0c872d484048f8f6b4d4
  Stored in directory: /root/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl size=180768 sha256=335e48a7c7699e871ce908cdc32e637b15ceabd5e2184a4f562c487e900fd842
  Stored in directory: /root/.cache/pip/wheels/3c/d8/9f/59fd78b2b7d1e9ffcb68fb6de80c2e7c20b804c8cbc4d8fc23
Successfully built nltk libwapiti
  Attempting unin

In [2]:
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm 
from torch import nn
import random
import torch
import hazm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DEVICE = torch.device('cpu')
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')

In [5]:
BASE_PATH = '/content/drive/MyDrive/DL/HW4/'
DATA_PATH = BASE_PATH+'data/'
BATCH_SIZE = 1
EPOCH_NUM = 1

In [6]:
random.seed(10)

##Data

In [7]:
!ls /content/drive/MyDrive/DL/HW4/data/

'ferdousi (1).txt'  'ferdousi (2).txt'	 ferdousi.txt   Persian_poems_corpus


In [8]:
poems_list = []
poem_file = DATA_PATH+'ferdousi.txt'
normalizer = hazm.Normalizer()
with open(poem_file) as file:
  lines = file.readlines()
  del lines[0]
  del lines[0]
  del lines[-1]
  mesras = []
  for line in lines:
    # beyt = lines[i].rstrip()+' '+lines[i+1].rstrip()
    mesra = line.rstrip()
    normalized_mesra = normalizer.normalize(mesra)
    mesras.append(normalized_mesra)
    # break
  data=[]
  for i in range(0,len(mesras)-1,2):
    data.append((mesras[i], mesras[i+1]))

In [10]:
tokenizer = hazm.word_tokenize
word2index = {}
index2word = {0: "SOS", 1: "EOS"}
VOCAB_SIZE = 2  
for mesra in mesras:
    tokens = tokenizer(mesra)
    # words = beyt.split(' ')
    for word in tokens:
        if word not in word2index:
            word2index[word] = VOCAB_SIZE
            index2word[VOCAB_SIZE] = word
            VOCAB_SIZE += 1

In [11]:
print(word2index)

{'به': 2, 'نام': 3, 'خداوند': 4, 'جان': 5, 'و': 6, 'خرد': 7, 'کزین': 8, 'برتر': 9, 'اندیشه': 10, 'برنگذرد': 11, 'جای': 12, 'روزی': 13, 'ده': 14, 'رهنمای': 15, 'کیوان': 16, 'گردان': 17, 'سپهر': 18, 'فروزنده': 19, 'ماه': 20, 'ناهید': 21, 'مهر': 22, 'ز': 23, 'نشان': 24, 'گمان': 25, 'برترست': 26, 'نگارندهٔ': 27, 'بر': 28, 'شده': 29, 'پیکرست': 30, 'بینندگان': 31, 'آفریننده': 32, 'را': 33, 'نبینی': 34, 'مرنجان': 35, 'دو': 36, 'بیننده': 37, 'نیابد': 38, 'بدو': 39, 'نیز': 40, 'راه': 41, 'که': 42, 'او': 43, 'از': 44, 'جایگاه': 45, 'سخن': 46, 'هر': 47, 'چه': 48, 'زین': 49, 'گوهران': 50, 'بگذرد': 51, 'خرد\u200cگر': 52, 'برگزیند': 53, 'همی': 54, 'همان': 55, 'گزیند': 56, 'بیند': 57, 'ستودن': 58, 'نداند': 59, 'کس': 60, 'چو': 61, 'هست': 62, 'میان': 63, 'بندگی': 64, 'ببایدت': 65, 'بست': 66, 'سنجد': 67, 'اوی': 68, 'در': 69, 'اندیشهٔ': 70, 'سخته': 71, 'کی': 72, 'گنجد': 73, 'بدین': 74, 'آلت': 75, 'رای': 76, 'زبان': 77, 'ستود': 78, 'توان': 79, 'هستیش': 80, 'باید': 81, 'خستو': 82, 'شوی': 83, 'گفتار': 84, '

In [12]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

In [18]:
from torch.utils.data import Dataset

class PoemDataset(Dataset):
    def __init__(self, data, word2index):
        self.data = data
        self.word2index = word2index
        # self.normalizer = hazm.Normalizer()
        self.tokenizer = hazm.word_tokenize

    def indexesFromSentence(self, sentence):
        tokens = self.tokenizer(sentence)
        # return [self.word2index[word] for word in sentence.split(' ')]
        return [self.word2index[word] for word in tokens]

    def tensorFromSentence(self, sentence):
        indexes = self.indexesFromSentence(sentence)
        indexes.append(EOS_token)
        return torch.tensor(indexes, dtype=torch.long, device=DEVICE)

    def __getitem__(self, index):   
        x = self.tensorFromSentence(data[index][0])
        y = self.tensorFromSentence(data[index][1]).view(-1,1)
        return (x,y)
    
    def __len__(self):
        return len(self.data)

In [19]:
total_data_size = len(data)
train_size = int(0.9 * total_data_size)
test_size = total_data_size-train_size

poem_dataset = PoemDataset(data, word2index)

train_dataset, test_dataset = torch.utils.data.random_split(poem_dataset, [train_size, test_size])

##Model

In [15]:
class Encoder(nn.Module):
   def __init__(self, hidden_state_size, embedding_size, num_layers):
       super(Encoder, self).__init__()
       self.embedding = nn.Embedding(VOCAB_SIZE, embedding_size)
       self.gru = nn.GRU(embedding_size, hidden_state_size, num_layers=num_layers, bidirectional=True)
              
   def forward(self, first_beyt):
       embedding = self.embedding(first_beyt).unsqueeze(1)
       outputs, hidden_state = self.gru(embedding)
       return outputs, hidden_state

class Decoder(nn.Module):
   def __init__(self, hidden_state_size, embedding_size, num_layers):
       super(Decoder, self).__init__()
       self.embedding = nn.Embedding(VOCAB_SIZE, embedding_size)
       self.gru = nn.GRU(embedding_size, hidden_state_size, num_layers=num_layers, bidirectional=True)
       self.out = nn.Linear(2*hidden_state_size, VOCAB_SIZE)
       self.softmax = nn.LogSoftmax(dim=1)
      
   def forward(self, input, hidden_state):
       input = input.view(1, -1)
       embedding = F.relu(self.embedding(input))
       output, hidden_state = self.gru(embedding, hidden_state) 
       logits = self.out(output[0])
       prediction = self.softmax(logits)
       return prediction, hidden_state

class BeytSora(nn.Module):
   def __init__(self, hidden_state_size, embedding_size, num_layers):
       super().__init__()      
       self.teacher_forcing_ratio = 0.5
       self.encoder = Encoder(hidden_state_size, embedding_size, num_layers)
       self.decoder = Decoder(hidden_state_size, embedding_size, num_layers)
     
   def forward(self, source, target):
       target_length = target.shape[0]
       outputs = torch.zeros(target_length, BATCH_SIZE, VOCAB_SIZE).to(DEVICE)

       encoder_output, encoder_hidden = self.encoder(source)
       decoder_hidden = encoder_hidden.to(DEVICE)
       decoder_input = torch.tensor([SOS_token], device=DEVICE)
       for t in range(target_length):   
           decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
           outputs[t] = decoder_output
           teacher_force = (random.random() < 0.5) and model.training
           topv, topi = decoder_output.topk(1)
           decoder_input = (target[t] if teacher_force else topi)
           if(teacher_force == False and decoder_input.item() == EOS_token):
               break

       return outputs

In [16]:
def get_sentence_loss(output, y, loss_fn):
  sentence_length = output.size(0)
  loss = 0
  for i in range(sentence_length):
    loss += loss_fn(output[i], y[i])
  return loss , loss.item()/sentence_length

def train_loop(model, train_dataset):
    loss_list = []
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=0.0002) 
    loss_fn = nn.CrossEntropyLoss()
    for epoch in range(0, EPOCH_NUM):
      print('-'*10, epoch, '-'*10)
      total_loss = 0
      for x,y in tqdm(train_dataset):
        optimizer.zero_grad()
        output = model(x, y)
        loss, loss_value = get_sentence_loss(output, y, loss_fn)
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 6)
        optimizer.step()
        total_loss += loss_value
      print('train_loss', total_loss)
      loss_list.append(total_loss)
      model_path = '/content/drive/MyDrive/DL/HW4/gru/'+f'{epoch}.pt'
      torch.save(model, model_path)
    return model

In [21]:
model = BeytSora(1024, 1024, 1).to(DEVICE)
# model =torch.load('/content/drive/MyDrive/DL/HW4/gru/0.pt')

model = train_loop(model, train_dataset)

---------- 0 ----------


100%|██████████| 44647/44647 [42:44<00:00, 17.41it/s]


train_loss 329830.6752048201


In [ ]:
def get_beyt(embedding):
  embedding = torch.flatten(embedding)
  beyt=[]
  for index in embedding.cpu().numpy():
    beyt.append(index2word[index])
  return beyt

In [ ]:
model =torch.load('/content/drive/MyDrive/DL/HW4/gru/0.pt')
model.eval()
for i in range(10):
  random_index = random.randint(0,len(test_dataset))
  x,y=test_dataset[random_index]
  logits = model(x, y)
  output = torch.argmax(logits, dim=2)
  beyt = get_beyt(output)
  print('first beyt :', get_beyt(x.view(-1,1)))
  print('second beyt :', get_beyt(y))
  print('predicted beyt :', beyt)

predicted beyt : ['چو', 'گفت', 'و', 'و', 'و', 'ز', 'ز', 'و', 'و', 'و', 'و', 'EOS', 'SOS', 'SOS', 'SOS', 'SOS', 'SOS']
predicted beyt : ['چو', 'گفت', 'کای', 'شهریار', 'که', 'ز', 'و', 'و', 'و']
predicted beyt : ['چو', 'گفت', 'و', 'و', 'و', 'که', 'که', 'که', 'از', 'و', 'و', 'EOS']
predicted beyt : ['چو', 'گفت', 'و', 'و', 'و', 'ز', 'که', 'ز', 'هر', 'و', 'و', 'EOS', 'SOS']
predicted beyt : ['چو', 'گفت', 'و', 'و', 'و', 'ز', 'که', 'که', 'و', 'و', 'و']
predicted beyt : ['چو', 'گفت', 'و', 'از', 'به', 'و', 'ز', 'که', 'و', 'و', 'و', 'EOS']
predicted beyt : ['چو', 'گفت', 'و', 'از', 'به', 'و', 'به', 'به', 'و', 'و', 'EOS']
predicted beyt : ['چو', 'گفت', 'و', 'و', 'و', 'ز', 'که', 'که', 'و', 'و']
predicted beyt : ['چو', 'گفت', 'که', 'از', 'تو', 'را', 'که', 'ز']
predicted beyt : ['چو', 'گفت', 'و', 'و', 'و', 'ز', 'که', 'که', 'و', 'و', 'و', 'EOS']
